# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 67 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.12821


extracting tarball to tmp_2208.12821...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.12825


extracting tarball to tmp_2208.12825...

...

/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.13668/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2208.13670...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.13749


extracting tarball to tmp_2208.13749...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13218-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13218) | **Kinematic data rebuild the Nuclear star cluster as the most metal rich  region of the Galaxy**  |
|| <mark>Francisco Nogueras-Lara</mark> |
|*Appeared on*| *2022-08-30*|
|*Comments*| *Accepted for publication in A&A. 8 pages, 8 figures*|
|**Abstract**| The Galactic centre (GC) is located at only 8 kpc from Earth and constitutes a unique template to understand Galactic nuclei. Nevertheless, the high crowding and extinction towards the GC hamper the study of its main stellar components, the nuclear stellar disc (NSD) and the nuclear star cluster (NSC). Recent work has suggested that the NSD and the NSC can be distinguished along the line of sight towards the NSC via the different extinction of their stars. This motivated us to analyse the proper motion, radial velocity, and the metallicity distributions of the different extinction groups. We use photometric, kinematic, and metallicity data to distinguish between probable NSD and NSC stars in a region centred on the NSC. We detected two different extinction groups of stars and obtained significantly different proper motion distributions for each of them, in agreement with the expected kinematics for the NSD and the NSC. We derived radial velocity maps that appear to be different for the NSD and the NSC. We also found different metallicities for each of the components, with the largest one measured for the most extinguished group of stars. We obtained that the metallicity distribution of each extinction group is best fitted by a bimodal distribution, indicating the presence of two metallicity components for each of them (a broad one slightly below solar metallicity, and a more metal rich narrower one, that is largest for the high extinction group of stars). We conclude that both extinction groups are distinct GC components with different kinematics and metallicity, and correspond to the NSD and the NSC. Therefore, it is possible to distinguish them via their different extinction. The high mean metallicity, $[M/H]\sim0.3$\,dex, obtained for the NSC metal rich stars, supports that the NSC is arguabily the most metal rich region of the Galaxy. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12821-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12821) | **Multiple locations of near-infrared coronal lines in NGC 5548**  |
|| Daniel Kynoch, et al. -- incl., <mark>Hermine Landt</mark> |
|*Appeared on*| *2022-08-30*|
|*Comments*| *20 pages, 16 figures and 6 tables. Accepted for publication in MNRAS*|
|**Abstract**| We present the first intensive study of the variability of the near-infrared coronal lines in an active galactic nucleus (AGN). We use data from a one-year long spectroscopic monitoring campaign with roughly weekly cadence on NGC 5548 to study the variability in both emission line fluxes and profile shapes. We find that in common with many AGN coronal lines, those studied here are both broader than the low-ionisaton forbidden lines and blueshifted relative to them, with a stratification that implies an origin in an outflow interior to the standard narrow line region. We observe for the first time [S VIII] and [Si VI] coronal line profiles that exhibit broad wings in addition to narrow cores, features not seen in either [S IX] or [Si X]. These wings are highly variable, whereas the cores show negligible changes. The differences in both the profile shapes and variability properties of the different line components indicate that there are at least two coronal line regions in AGN. We associate the variable, broad wings with the base of an X-ray heated wind evaporated from the inner edge of the dusty torus. The coronal line cores may be formed at several locations interior to the narrow line region: either along this accelerating, clumpy wind or in the much more compact outflow identified with the obscurer and so emerging on scales similar to the outer accretion disc and broad line region. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12825-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12825) | **Uncovering dark matter density profiles in dwarf galaxies with graph  neural networks**  |
|| Tri Nguyen, <mark>Siddharth Mishra-Sharma</mark>, <mark>Reuel Williams</mark>, Lina Necib |
|*Appeared on*| *2022-08-30*|
|*Comments*| *9 + 11 pages, 4 + 9 figures*|
|**Abstract**| Dwarf galaxies are small, dark matter-dominated galaxies, some of which are embedded within the Milky Way. Their lack of baryonic matter (e.g., stars and gas) makes them perfect test beds for probing the properties of dark matter -- understanding the spatial dark matter distribution in these systems can be used to constrain microphysical dark matter interactions that influence the formation and evolution of structures in our Universe. We introduce a new method that leverages simulation-based inference and graph-based machine learning in order to infer the dark matter density profiles of dwarf galaxies from observable kinematics of stars gravitationally bound to these systems. Our approach aims to address some of the limitations of established methods based on dynamical Jeans modeling. We show that this novel method can place stronger constraints on dark matter profiles and, consequently, has the potential to weigh in on some of the ongoing puzzles associated with the small-scale structure of dark matter halos, such as the core-cusp discrepancy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13749-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13749) | **Rapid Blue- and Red-shifted Excursions in H$α$ line profiles  synthesized from realistic 3D MHD simulations**  |
|| S. Danilovic, J. P. Bjørgen, <mark>J. Leenaarts</mark>, M. Rempel |
|*Appeared on*| *2022-08-30*|
|*Comments*| *submitted to A&A, movies in this https URL*|
|**Abstract**| Rapid blue- and red-shifted events (RBEs/RREs) may have an important role in mass-loading and heating the solar corona, but their nature and origin are still debatable. We aim to model these features to learn more about their properties, formation and origin. A realistic three-dimensional (3D) magneto-hydrodynamic (MHD) model of a solar plage region is created. Synthetic H$\alpha$ spectra are generated and the spectral signatures of these features are identified. The magnetic field lines associated with these events are traced and the underlying dynamic is studied. The model reproduces well many properties of RBEs and RREs, such as spatial distribution, lateral movement, length and lifetimes. Synthetic H$\alpha$ line profiles, similarly to observed ones, show strong blue- or red-shift and asymmetries. These line profiles are caused by the vertical component of velocity with magnitudes larger than $30-40$ km/s that appear mostly in the height range of $2-4$ Mm. By tracing magnetic field lines, we show that the vertical velocity that causes the appearance of RBE/RREs to appear is always associated with the component of velocity perpendicular to the magnetic field line. The study confirms the hypothesis that RBEs and RREs are signs of Alfv{\'e}nic waves with, in some cases, a significant contribution from slow magneto-acoustic mode. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13511-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13511) | **Galaxies with Fuzzy Dark Matter**  |
|| <mark>Jae-Weon Lee</mark> |
|*Appeared on*| *2022-08-30*|
|*Comments*| **|
|**Abstract**| This is a brief review on some properties of galaxies in the fuzzy dark matter model, where dark matter is an ultra-light scalar particle with mass $m = O(10^{-22})eV$. From quantum pressure, dark matter has a halo length scale which can solve the small scale issues of the cold dark matter model, such as the core-cusp problem, and explain many other observed mysteries of galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13463-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13463) | **Field of View and contrast limitations of stellar interferometers. A  quick review**  |
|| <mark>Francois Henault</mark> |
|*Appeared on*| *2022-08-30*|
|*Comments*| **|
|**Abstract**| Field of View (FoV) and contrast limitations of stellar interferometers have been the scope of numerous publications for more than thirty years. Recently, this topic regained some interest since long-baseline terrestrial interferometers or space borne nulling interferometers are envisioned for detecting and characterizing extra-solar planets orbiting in the habitable zone of their parent star. This goal supposes to achieving sufficient contrast ratio in the high angular frequency domain, thus on the whole interferometer FoV. In this paper are reviewed some of the contrast and FoV limiting factors, including spectral bandwidth, flux mismatches, fringe tracking, telescope image quality, atmosphere seeing, optical conjugation mismatch of the telescopes pupils, influence of anamorphous optics, pupil aberrations, signal-to-noise ratio and deviations with respect to the golden rule of imaging interferometers. Finally, a tentative classification of all these factors is provided. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13470-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13470) | **Nulling interferometry in space does not require a rotating telescope  array**  |
|| <mark>Francois Henault</mark> |
|*Appeared on*| *2022-08-30*|
|*Comments*| **|
|**Abstract**| Space borne nulling interferometry in the mid-infrared waveband is one of the most promising techniques for characterizing the atmospheres of extra-solar planets orbiting in the habitable zone of their parent star, and possibly discovering life markers. One of its most difficult challenges is the control of free-flying telescope spacecrafts moving around a central combiner in order to modulating the planet signal, within accuracy better than one micrometer at least. Moreover, the whole array must be reconfigured regularly in order to observe different celestial targets, thus increasing the risk of loosing one or more spacecrafts and aborting the mission before its normal end. In this paper is described a simplified optical configuration where the telescopes do not need to be rotated, and the number of necessary array reconfigurations is minimized. It allows efficient modulation of the planet signal, only making use of rotating prisms or mirrors located into the central combiner. In this paper the general principle of a nulling interferometer with a fixed telescope array is explained. Mathematical relations are established in order to determining the planet modulation signal. Numerical simulations are carried out for three different arrangements of the collecting telescopes. They confirm that nulling interferometry in space does not require a rotating telescope array |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13482-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13482) | **Lord of the Ring Gratings: Their use as image plane filters for  coronagraphy**  |
|| <mark>Francçois Henault</mark> |
|*Appeared on*| *2022-08-30*|
|*Comments*| **|
|**Abstract**| Coronagraphy is an efficient technique for identifying and characterizing extra-solar planets orbiting in the habitable zone of their parent star. An important family of coronagraphs is based on amplitude or phase filters placed at an intermediate image plane of the optical system, spreading starlight outside of the so-called Lyot stop located at the exit pupil plane of the instrument. This article explores the potential of circular amplitude and phase gratings employed as image plane coronagraph filters. It presents a theoretical analysis of the simplest case of an amplitude circular grating and introduces an inversion paradigm with respect to classical Lyot coronagraph, by exchanging its image and pupil masks. Various types of circular gratings are considered and their performance is evaluated with the help of numerical simulations. The most promising solutions are presented. We conclude that high attenuation ratios of the starlight are feasible, provided that the system has been carefully optimized |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.13218.md
    + _build/html/tmp_2208.13218/./uncertainties.png
    + _build/html/tmp_2208.13218/./CMD.png
    + _build/html/tmp_2208.13218/./Fig1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$</div>



<div id="title">

# Kinematic data rebuild the Nuclear star cluster as the most metal rich region of the Galaxy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.13218-b31b1b.svg)](https://arxiv.org/abs/2208.13218)<mark>Appeared on: 2022-08-30</mark> - _Accepted for publication in A&A. 8 pages, 8 figures_

</div>
<div id="authors">

<mark>F. Nogueras-Lara</mark>

</div>
<div id="abstract">

**Abstract:** The Galactic centre (GC) is located at only 8 kpc from Earth and constitutes a unique template to understand Galactic nuclei. Nevertheless, the high crowding and extinction towards the GC hamper the study of its main stellar components, the nuclear stellar disc (NSD) and the nuclear star cluster (NSC).Recent work has suggested that the NSD and the NSC can be distinguished along the line of sight towards the NSC via the different extinction of their stars. This motivated us to analyse the proper motion, radial velocity, and the metallicity distributions of the different extinction groups.We use photometric, kinematic, and metallicity data to distinguish between probable NSD and NSC stars in a region centred on the NSC.We detected two different extinction groups of stars and obtained significantly different proper motion distributions for each of them, in agreement with the expected kinematics for the NSD and the NSC. We derived radial velocity maps that appear to be different for the NSD and the NSC. We also found different metallicities for each of the components, with the largest one measured for the most extinguished group of stars. We obtained that the metallicity distribution of each extinction group is best fitted by a bimodal distribution, indicating the presence of two metallicity components for each of them (a broad one slightly below solar metallicity, and a more metal rich narrower one, that is largest for the high extinction group of stars).We conclude that both extinction groups are distinct GC components with different kinematics and metallicity, and correspond to the NSD and the NSC. Therefore, it is possible to distinguish them via their different extinction. The high mean metallicity,$[M/H]\sim0.3$dex, obtained for the NSC metal rich stars, supports that the NSC is arguabily the most metal rich region of the Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2208.13218/./uncertainties.png" alt="Fig2" width="100%"/>

**Figure 2. -** Colour-magnitude diagram $K_s$ versus $J-K_s$ of common stars between the photometric and the proper motion catalogues. The colour code corresponds to stellar densities using a power stretch scale. The blue dashed boxes indicate the stars belonging to the NSD and the NSC following the different extinction criterion. Right panels show the uncertainties of the proper motions $\mu_l$(Galactic longitude coordinate) and $\mu_b$(Galactic latitude coordinate). The blue dashed lines indicate the 90th percentile of the data (that is below 0.7 mas/yr in both cases). (*uncer*)

</div>
<div id="div_fig2">

<img src="tmp_2208.13218/./CMD.png" alt="Fig1" width="100%"/>

**Figure 1. -** Colour-magnitude diagram $K_s$ versus $J-K_s$. The colour code corresponds to stellar densities using a power stretch scale, where the number of stars per bin is indicated with the colour bar. The blue dashed lines show the division between foreground, NSD, and NSC stars. The numbers 1 and 2 indicate the red clump bumps with different extinction that belong to the NSD, and the NSC, respectively. (*CMD*)

</div>
<div id="div_fig3">

<img src="tmp_2208.13218/./Fig1.png" alt="Fig8" width="100%"/>

**Figure 8. -** GALACTICNUCLEUS false-colour image of the analysed region using $JHK_s$ bands. The white dashed line shows the effective radius of the NSC. White shaded contours indicates regions where there is not coverage from the proper motion catalogue. The dotted contour shows the region with stars with known metallicities. (*GNS*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

21  publications in the last 7 days.
	 _build/html/2208.13218.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers